In [1]:
import wandb
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns, matplotlib.pyplot as plt, operator as op


In [2]:
api = wandb.Api(timeout=60)

In [3]:
all_runs_data = {}
agents = ['MDQN_modern', 'DQN_modern'] # DQN_modern
environments = ['Qbert', 'BattleZone', 'DoubleDunk', 'NameThisGame', 'Phoenix']
checkpoints = ['model_15250000', 'model_37250000', 'model_50000000']
networks = [('nature', 0.001), 
            ('dueling', 0.001),
            ('impala_large:1', 0.0005),
            ('impala_large:2', 0.0005),
            ('impala_large:4', 0.0001),
            # ('impalanextv2_large:1', 0.0001),
            ('impalanextv2_large:2', 0.00001)]

for agent in agents:
    for environment in environments:
        for checkpoint in checkpoints:
            base_mae = None
            for model, lr in networks:

                project_name = f'benchmarkv2_final_{model}_{agent}_{environment}_{checkpoint}_0_{model}'.replace(':', '_')
                try:
                    runs = api.runs('autoconvnext/' + project_name)
                    all_runs_data[project_name] = []
                    maes = []
                    for run in runs:
                        
                        if model == 'impalanextv2_large:2':
                            # print(run.config['lr'])
                            if run.config['lr'] == 0.0001:
                                continue
                        all_runs_data[project_name].append(run.history())
                except Exception as e:
                    print(project_name, e)

In [4]:
import numpy as np
np.save('all_runs_datav2.npy', all_runs_data)
# all_runs_data = np.load('all_runs_data.npy', allow_pickle=True).item()

In [11]:
all_runs_data['benchmarkv2_final_nature_MDQN_modern_Qbert_model_15250000_0_nature'][0]['test_mae'].to_numpy()

array([5.27255058, 5.20228291, 5.02153015, 4.47402906, 3.889642  ,
       3.61217523, 3.49167228, 3.40863705, 3.35061884, 3.33205819,
       3.26230693, 3.25576854, 3.23489165, 3.2414968 , 3.21473598,
       3.20268011, 3.21393347, 3.28497672, 3.22716188, 3.18372178,
       3.18814611, 3.21714687, 3.18219805, 3.17025638, 3.16594982,
       3.1888237 , 3.23869705, 3.2363565 , 3.23087358, 3.24912858])

In [12]:
def get_vals(agents, environments, checkpoints, networks):
    model_maes = {
        'dueling': [],
        'nature': [],
        'impala_large:1': [],
        'impala_large:2': [],
        'impala_large:4': [],
        # 'impalanextv2_large:1': [],
        'impalanextv2_large:2': [] # TODO check for the 2 the LR, there was a bug and we run it with two lrs
        }
    
    model_stds = {
        'dueling': [],
        'nature': [],
        'impala_large:1': [],
        'impala_large:2': [],
        'impala_large:4': [],
        # 'impalanextv2_large:1': [],
        'impalanextv2_large:2': [] # TODO check for the 2 the LR, there was a bug and we run it with two lrs
        }
    model_improvements = {
        'dueling': [],
        'nature': [],
        'impala_large:1': [],
        'impala_large:2': [],
        'impala_large:4': [],
        # 'impalanextv2_large:1': [],
        'impalanextv2_large:2': [] # TODO check for the 2 the LR, there was a bug and we run it with two lrs
        }
    i = 0
    for agent in agents:
        for environment in environments:
            for checkpoint in checkpoints:
                base_mae = None
                for model, lr in networks:

                    project_name = f'benchmarkv2_final_{model}_{agent}_{environment}_{checkpoint}_0_{model}'.replace(':', '_')
                    # runs = api.runs('autoconvnext/' + project_name)
                    runs = all_runs_data[project_name]
                    maes = []
                    succ = 0
                    for run in runs:
                        try:
                            validations_maes = run['validation_mae'].to_numpy()
                            test_maes = run['test_mae'].to_numpy()

                            final_mae = test_maes[np.argmin(validations_maes)]
                            # if model == 'impalanextv2_large:2':
                            #     # print(run.config['lr'])
                            #     if run.config['lr'] == 0.0001:
                            #         continue
                            # maes.append(run.iloc[29]['test_mae'])
                            maes.append(final_mae)
                            succ += 1
                            if succ == 10:
                                break
                        except Exception as e:
                            print(project_name, e)
                    if succ < 10:
                        print(project_name, succ)
                    # print(project_name, pd.Series(maes).mean())
                    model_maes[model].append(pd.Series(maes).mean())
                    model_stds[model].append(pd.Series(maes).std())

                    if model == 'nature':
                        base_mae = pd.Series(maes).mean()
                    model_improvements[model].append((base_mae - pd.Series(maes).mean()) / base_mae * 100)
                    i += 1

                    if i % 10 == 0:
                        print(i)

    all = []
    for model, improvements in model_improvements.items():
        print(model, pd.Series(improvements).mean(), pd.Series(improvements).std())
        all.append(np.array(improvements))
    return model_maes, model_stds, model_improvements, all

    # sns.swarmplot(data=sorted_vals, size=6, edgecolor="black", linewidth=.9)

In [13]:
def plot(model_improvements, name):
    del model_improvements['nature']
    sorted_keys, sorted_vals = zip(*sorted(model_improvements.items(), key=op.itemgetter(1)))
    correct_order = ['dueling', 'impala_large:1', 'impala_large:2', 'impala_large:4', 'impalanextv2_large:2']
    sorted_keys = [correct_order.index(k) for k in sorted_keys]
    sorted_vals = list(sorted_vals)
    sorted_keys = list(sorted_keys)

    sorted_vals = [x for _, x in sorted(zip(sorted_keys, sorted_vals))]

    # almost verbatim from question
    sns.set(rc={'figure.figsize':(5,3)})

    sns.set(context='notebook', style='whitegrid')
    # sns.axlabel(xlabel="Groups", ylabel="Y-Axis", fontsize=16)
    sns.boxplot(data=sorted_vals, width=.4, orient='v')
    plt.xticks(range(0, len(sorted_keys)), ['Dueling', 'IMPALA:1', 'IMPALA:2', 'IMPALA:4', 'I-NeXt'])
    plt.xlabel('Model')
    plt.ylabel('Improvement (%)')
    plt.ylim(-10, 20)
    plt.savefig(f'figures/{name}.png', bbox_inches='tight')
    plt.savefig(f'figures/{name}.pdf', bbox_inches='tight')
    plt.clf()

In [15]:
all_agents = ['MDQN_modern', 'DQN_modern']
all_environments = ['Qbert', 'BattleZone', 'DoubleDunk', 'NameThisGame', 'Phoenix']
all_checkpoints = ['model_15250000', 'model_37250000', 'model_50000000']
networks = [('nature', 0.001), 
            ('dueling', 0.001),
            ('impala_large:1', 0.0005),
            ('impala_large:2', 0.0005),
            ('impala_large:4', 0.0001),
            ('impalanextv2_large:2', 0.00001)]

In [6]:
get_vals(all_agents, all_environments, all_checkpoints, networks)[2]

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
dueling -3.1341414195341954 6.678998431798878
nature 0.0 0.0
impala_large:1 6.511299590757182 3.814507252637409
impala_large:2 5.429128031749615 3.784677506251962
impala_large:4 7.247351333059283 4.282215034795758
impalanextv2_large:2 4.125647082936263 5.137107006351065


{'dueling': [-0.08050910528412523,
  -1.848807274965423,
  -1.0383411174604997,
  0.5402923591543178,
  0.437199465078546,
  -0.48598477679356045,
  -2.8851975416762308,
  -1.4872136573966546,
  -0.534768151419593,
  -1.1113240361484333,
  -1.2764271804627183,
  -2.7370784867016624,
  -34.427625812558475,
  -15.708012631675308,
  -8.16773422592798,
  -0.025622447379926323,
  -2.7781798360176513,
  0.6387717693280601,
  -0.30262812104177067,
  0.14712526323787128,
  0.1749104914170555,
  -2.5294231369718223,
  -3.084909800128784,
  -1.7650746229329444,
  -1.7301644933602582,
  -1.4220128090409372,
  -1.3799160349613175,
  -3.8676554055332817,
  -2.832806757140761,
  -2.455124471261601],
 'nature': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'impala_large:1': [5.4826139685351185,
  4.056255414566816,
  -0.8257326336368637,


In [16]:
plot(get_vals(all_agents, all_environments, all_checkpoints, networks)[2], 'all')

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
dueling -0.47943300135890005 0.8794001984511027
nature 0.0 0.0
impala_large:1 3.7954540065226907 2.7315158411175258
impala_large:2 4.2100452553559995 2.961477091152762
impala_large:4 -17.935924352158743 23.222299433702315
impalanextv2_large:2 -21.56464757794471 18.979962121439637


<Figure size 500x300 with 0 Axes>

In [17]:
for agent in all_agents:
    plot(get_vals([agent], all_environments, all_checkpoints, networks)[2], agent)

10
20
30
40
50
60
70
80
90
dueling -0.5288708369564247 1.1495504380265602
nature 0.0 0.0
impala_large:1 3.362093591979436 3.0138696102426463
impala_large:2 3.9386210084353697 3.3613832547017273
impala_large:4 -23.851117006981926 30.114457796169926
impalanextv2_large:2 -25.395322714506488 23.617417224945388
10
20
30
40
50
60
70
80
90
dueling -0.42999516576137536 0.5246186805973259
nature 0.0 0.0
impala_large:1 4.228814421065946 2.443247357011218
impala_large:2 4.481469502276631 2.5904405752816144
impala_large:4 -12.02073169733556 11.62806782056422
impalanextv2_large:2 -17.733972441382935 12.529263438820173


<Figure size 500x300 with 0 Axes>

In [18]:
for environment in all_environments:
    plot(get_vals(all_agents, [environment], all_checkpoints, networks)[2], environment)

10
20
30
dueling -0.44605540933624793 0.554108837603578
nature 0.0 0.0
impala_large:1 2.9446870676300025 1.854403439670352
impala_large:2 4.091261363273735 0.8984796981497948
impala_large:4 -18.620644922207607 2.7809612012776
impalanextv2_large:2 -28.522415795825506 3.182113907597313
10
20
30
dueling -0.003963811498976878 0.24076033501217026
nature 0.0 0.0
impala_large:1 8.319154147356317 0.8155578492298243
impala_large:2 8.99284875546593 0.6687264219597492
impala_large:4 -3.380508878451168 1.8522442791037126
impalanextv2_large:2 -9.526097750764565 1.364765157514292
10
20
30
dueling -0.21303398493949452 0.29938422051121766
nature 0.0 0.0
impala_large:1 3.158177384829658 0.23838324833827707
impala_large:2 3.7237571400406018 0.4612677499421064
impala_large:4 -5.389170938330385 5.272030933897837
impalanextv2_large:2 -10.898320006292487 5.730676892246114
10
20
30
dueling 0.01278874397767952 0.4315266861398865
nature 0.0 0.0
impala_large:1 2.4313027859154115 1.5758966497887783
impala_large:

<Figure size 500x300 with 0 Axes>

In [19]:
for checkpoint in all_checkpoints:
    plot(get_vals(all_agents, all_environments, [checkpoint], networks)[2], checkpoint)

10
20
30
40
50
60
dueling -0.4332425544805577 0.7186062727720324
nature 0.0 0.0
impala_large:1 4.482672143756935 1.9790905691893947
impala_large:2 4.616946069676063 2.190600595412995
impala_large:4 -15.55278519436646 16.773556267107057
impalanextv2_large:2 -20.4332013081997 14.73974715067283
10
20
30
40
50
60
dueling -0.4620353296416978 0.7015933986677905
nature 0.0 0.0
impala_large:1 3.712637225456949 3.020705612646063
impala_large:2 3.8361060654568524 3.650140407335801
impala_large:4 -19.73404174395994 26.538414597308183
impalanextv2_large:2 -21.671172042563466 16.937170280365986
10
20
30
40
50
60
dueling -0.5430211199544448 1.214932337358766
nature 0.0 0.0
impala_large:1 3.1910526503541883 3.172043577832049
impala_large:2 4.177083630935085 3.130049591957856
impala_large:4 -18.520946118149833 27.234935687947473
impalanextv2_large:2 -22.589569383070966 25.574203854155762


<Figure size 500x300 with 0 Axes>

In [7]:
for agent in all_agents:
    for environment in all_environments:
        plot(get_vals(all_agents, [environment], all_checkpoints, networks)[2], f'{environment}_{agent}_only')

10
20
30
dueling -0.8554480019632608 1.283662238455703
nature 0.0 0.0
impala_large:1 3.3378076882551952 2.4737679766793392
impala_large:2 3.6276023778762974 2.5129719652922797
impala_large:4 5.306792181879019 1.456780085793692
impalanextv2_large:2 -3.680163609359005 1.9223067490070678
10
20
30
dueling 0.08515244684207658 0.40485999466908984
nature 0.0 0.0
impala_large:1 8.618802972258129 0.7219553064096135
impala_large:2 9.253393748364529 0.8985931128956193
impala_large:4 9.957539340129733 1.3135819160034348
impalanextv2_large:2 4.331739464979645 0.7290605966726746
10
20
30
dueling -2.0477644850876717 0.9687361126743478
nature 0.0 0.0
impala_large:1 9.601742164761637 0.3987074407405923
impala_large:2 8.123596953629464 0.736431874007008
impala_large:4 10.085571765843712 0.2562720122625462
impalanextv2_large:2 6.779445999872094 1.514575992226166
10
20
30
dueling -1.6094871734458878 0.5887223794784314
nature 0.0 0.0
impala_large:1 1.8076536769213682 1.4244094389236697
impala_large:2 0.566

<Figure size 500x300 with 0 Axes>

In [20]:
for agent in all_agents:
    for checkpoint in all_checkpoints:
        plot(get_vals(all_agents, all_environments, [checkpoint], networks)[2], f'{checkpoint}_{agent}_only')

10
20
30
40
50
60
dueling -0.4332425544805577 0.7186062727720324
nature 0.0 0.0
impala_large:1 4.482672143756935 1.9790905691893947
impala_large:2 4.616946069676063 2.190600595412995
impala_large:4 -15.55278519436646 16.773556267107057
impalanextv2_large:2 -20.4332013081997 14.73974715067283
10
20
30
40
50
60
dueling -0.4620353296416978 0.7015933986677905
nature 0.0 0.0
impala_large:1 3.712637225456949 3.020705612646063
impala_large:2 3.8361060654568524 3.650140407335801
impala_large:4 -19.73404174395994 26.538414597308183
impalanextv2_large:2 -21.671172042563466 16.937170280365986
10
20
30
40
50
60
dueling -0.5430211199544448 1.214932337358766
nature 0.0 0.0
impala_large:1 3.1910526503541883 3.172043577832049
impala_large:2 4.177083630935085 3.130049591957856
impala_large:4 -18.520946118149833 27.234935687947473
impalanextv2_large:2 -22.589569383070966 25.574203854155762
10
20
30
40
50
60
dueling -0.4332425544805577 0.7186062727720324
nature 0.0 0.0
impala_large:1 4.482672143756935 1.

<Figure size 500x300 with 0 Axes>